# Draft SNIa Host Science

(1) Choose a SNIa to explore. Use truth table.

(2) Show the SNIa environ (deep coadd).

(3) Explain how there will be 'nearby' galaxy Object IDs in the future. Cite the DMTN.

(4) Get Object table data for objects in environ.

(5) Identify three nearest stars and galaxies (radial), and three nearest extended objects (separation distance).

(6) External NED search for nearest low-z galaxy?


In [ ]:
import numpy
import matplotlib
import matplotlib.pyplot as plt

import pandas
pandas.set_option('display.max_rows', 1000)
from lsst.rsp import get_tap_service, retrieve_query

import lsst.daf.butler as dafButler
import lsst.geom
import lsst.afw.display as afwDisplay

afwDisplay.setDefaultBackend('matplotlib')

In [ ]:
butler = dafButler.Butler('dp02', collections='2.2i/runs/DP0.2')

In [ ]:
skymap = butler.get('skyMap')

In [ ]:
sn_ra = 67.4579
sn_dec = -44.0802
sn_spherePoint = lsst.geom.SpherePoint(sn_ra*lsst.geom.degrees,
                                       sn_dec*lsst.geom.degrees)
sn_tract = skymap.findTract(sn_spherePoint).tract_id
sn_patch = skymap.findTract(sn_spherePoint).findPatch(sn_spherePoint).getSequentialIndex()
sn_dataId_deepCoadd = {'band': 'i', 'tract': sn_tract, 'patch': sn_patch}
sn_deepCoadd = butler.get('deepCoadd', dataId=sn_dataId_deepCoadd)

In [ ]:
fig = plt.figure(figsize=(10, 8))
afw_display = afwDisplay.Display(1)
afw_display.scale('asinh', 'zscale')
afw_display.mtv(sn_deepCoadd.image)
plt.gca().axis('on')

In [ ]:
### NO -- USE TAP FOR CATALOGS
### meaning you must decide in advance which columns to use

# dataId = {'tract': sn_tract, 'patch': sn_patch}
# nearby_objects = butler.get('objectTable', dataId=dataId)

In [ ]:
# nearby_objects